In [1]:
import cv2
import matplotlib.pyplot as plt


# Importar imagens

In [2]:
img1 = cv2.imread("D1.BMP")
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

img2 = cv2.imread("N1.bmp")
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

In [27]:
# plt.imshow(img1)
# plt.imshow(img2)

# Conversão para Preto e Branco

In [3]:
img1_bw = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img2_bw = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

In [12]:
# plt.imshow(img1_bw, cmap="gray")
# plt.imshow(img2_bw, cmap="gray")

# Realçar contornos
(membranas celulares e seus núcleos)

## Constrate

In [28]:
# TODO

## Brilho

In [29]:
# TODO

## Filtros

In [32]:
# TODO

# Segmentar Núcleo e Citoplasma

In [33]:
# TODO